In [ ]:
def load_data(file_path, seq_len, batch_size, shuffle=True):
    # Read the data from CSV file
    df = pd.read_csv(file_path)
    
    # Normalize the data
    scaler = MinMaxScaler()
    data = scaler.fit_transform(df.values)
    
    # Create the dataset
    dataset = TimeSeriesDataSet(data, seq_len)
    
    # Create DataLoader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    
    return dataloader, scaler

# To experiment with some deep learning models on the time series data given

In [ ]:
!pip install torch

In [79]:
import torch
from torch import nn
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader
import numpy as np

x = torch.rand(1,2)
print(x)

tensor([[0.8038, 0.8045]])


In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_rate):
        super(LSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Define the fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)

        # add dropout layer
        self.dropout = nn.Dropout(dropout_rate)

        
    
    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # apply the dropout layer
        x = self.dropout(x)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
num_epochs = 10
input_size = 1
hidden_size = 50
num_layers = 2
output_size = 1
dropout_p = 0.2
learning_rate = 0.01

model = LSTM(
    input_size,
    hidden_size,
    num_layers,
    output_size,
    dropout_p
)

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [91]:
from torch.utils.data import Dataset
import pandas as pd

class TimeSeriesDataSet(Dataset):
    def __init__(self, csv_path):
        super().__init__()
        self.data = pd.read_csv(csv_path)
        # scaler = MinMaxScaler()
        # self.data = scaler.fit_transform(self.data.loc[:, 'Close'].values.reshape(1, -1)) # becomes numpy array
        self.data = self.data.loc[:, 'Close'].values.reshape(-1, 1)
        self.seq_len = len(self.data)

    def __len__(self):
        return self.seq_len
    
    def __getitem__(self, idx):
        features = self.data[idx, :-1]
        output = self.data[idx, -1]
        return features, output 


def load_data(file_path, batch_size, shuffle=False):
    # Create the dataset
    dataset = TimeSeriesDataSet(file_path)
    
    # Create DataLoader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    
    return dataloader

In [63]:

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    outputs = model(X_train)

    # compute loss function
    loss = criterion(outputs, y_train)

    # backpropagate the loss values
    loss.backward()

    # weight update
    optimizer.step()

    # disable gradient computations for test set
    with torch.no_grad():
        # compute test data loss and stuff i guess
        pass

NameError: name 'X_train' is not defined

In [92]:
dataloader = load_data('data/processed/mastercard_stock_history_processed.csv', 500)
df = pd.read_csv('data/processed/mastercard_stock_history_processed.csv')
df.head()


,Date,Close
0,2006-05-25,1.453770
1,2006-05-26,1.430235
2,2006-05-30,1.409318
3,2006-05-31,1.430457
4,2006-06-01,1.486069


In [95]:
for i, d in enumerate(dataloader):
    print(i)

0
1
2
3
4
5
6
7
